In [21]:
import os
import json
from dotenv import load_dotenv
from tqdm import tqdm
import pandas as pd
import numpy as np
import random
from scipy.stats import mannwhitneyu
import torch

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

load_dotenv()

True

In [22]:
if torch.backends.mps.is_available():  # For macOS devices
    device = torch.device("mps")
elif torch.cuda.is_available():  # For NVIDIA GPUs
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: mps


In [23]:
FIELDS_TO_CHECK = [
    "species",
    "strain",
    "gender",
    "treatment",
    "way_of_administration",
    "dosage",
    "age_at_start",
    "duration_unit",
    "median_treatment",
    "max_treatment",
    "median_control",
    "max_control",
    "p_value",
    "n_treatment",
    "n_control",
]

In [18]:
def create_reference_files(path_to_reference_data: str, folder_to_save: str):
    """
    Create reference files in the required format. path_to_reference_data files should have "doi" columns and FIELDS_TO_CHECK columns
    Args:
        path_to_reference_data (str): Path to the reference data Excel file.
    """
    os.makedirs(folder_to_save, exist_ok=True)
    reference_data = pd.read_excel(path_to_reference_data)
    reference_data = reference_data.rename(columns={"intervention": "treatment"})
    for doi in reference_data["doi"]:
        name = doi.split("/")[1]
        temp_df = reference_data[reference_data["doi"] == doi]
        output = {"groups": temp_df[FIELDS_TO_CHECK].to_dict(orient="records")}
        with open(f"{folder_to_save}/{name}.json", "w") as outfile:
            json.dump(output, outfile, indent=4)

In [ ]:
def calculate_simmilarity_scores(
    reference_data_folder: str,
    pipeline_result_folder: str,
    model: SentenceTransformer,
    control=False,
):
    """
    Calculate simmilarity scores between the pipeline answer and reference answer
    Args:
        reference_data_folder (str): Path to the folder containing reference data.
        pipeline_result_folder (str): Path to the folder containing pipeline results.
        model: SentenceTransformer model for computing embeddings.
        control (bool): If True, use random file for control, otherwise use exact file.
    Returns:
        List[float]: List of similarity scores between the pipeline answer and reference answer.
    """
    full_scores = []
    # for each doi we calculate the simmilarity scores between the pipeline answer and reference answer
    for i in tqdm(os.listdir(reference_data_folder)):
        reference_path = os.path.join(reference_data_folder, i)
        # for control we use random file, but for not control we use exact file
        if control:
            file = random.choice(
                [x for x in os.listdir(pipeline_result_folder) if x != ".DS_Store"]
            )
        else:
            file = i
        answer_path = os.path.join(pipeline_result_folder, file)
        reference_answer = json.load(open(reference_path, "r"))
        answer_answer = json.load(open(answer_path, "r"))
        scores = []
        reference_len = len(reference_answer["groups"])
        answer_len = len(answer_answer["groups"])
        # group_similarity is a penalty for discrepancy in the number of allocated experimental groups
        group_similarity = 1 / (1 + abs(answer_len - reference_len) / reference_len)
        # computing simmilarity score for each experimental group
        for answer in reference_answer["groups"]:
            ref = " ".join([str(answer[fld]) for fld in FIELDS_TO_CHECK])
            ref_embed = model.encode(ref)
            groups = answer_answer["groups"]
            answers = []
            for res in groups:
                res_temp = [str(res[fld]) for fld in FIELDS_TO_CHECK]
                answers.append(" ".join(res_temp))
            res_embed = model.encode(answers)
            # as we dont know the exact pair that we need to compare we took maximal similarity score across all experimental groups
            test = max([cosine_similarity([ref_embed], [x])[0][0] for x in res_embed])
            scores.append(test)
        metric = np.mean(scores) * group_similarity
        print(metric)
        full_scores.append(metric)
    return full_scores

In [19]:
create_reference_files("CollidaData_2023.xlsx",
                       "pipeline_results/reference_data")

In [24]:
random.choice([x for x in os.listdir("pipeline_results/openai") if x!='.DS_Store'])

'acel.13088.json'

In [27]:
model = SentenceTransformer("all-mpnet-base-v2",device=device)

In [29]:
control_scores = calculate_simmilarity_scores(reference_data_folder="pipeline_results/reference_data",
                             pipeline_result_folder="pipeline_results/openai",
                             model=model,
                             control=True)

  2%|▏         | 1/59 [00:00<00:10,  5.33it/s]

0.055029794573783875


  3%|▎         | 2/59 [00:02<01:23,  1.47s/it]

0.18383451278616741


  5%|▌         | 3/59 [00:02<00:52,  1.06it/s]

...

0.17841757759451868


 97%|█████████▋| 57/59 [00:35<00:02,  1.15s/it]

0.2041575452114673
0.23657187819480896
0.7486936450004578


100%|██████████| 59/59 [00:35<00:00,  1.68it/s]

0.16963714361190796


In [30]:
pipeline_scores = calculate_simmilarity_scores(reference_data_folder="pipeline_results/reference_data",
                             pipeline_result_folder="pipeline_results/openai",
                             model=model,
                             control=False)

  2%|▏         | 1/59 [00:00<00:20,  2.78it/s]

0.23264575004577637


  5%|▌         | 3/59 [00:02<00:47,  1.17it/s]

0.3291767468819251
0.5959968566894531
0.8674309849739075


  8%|▊         | 5/59 [00:02<00:22,  2.39it/s]

...

 93%|█████████▎| 55/59 [00:28<00:03,  1.07it/s]

0.29744039803016475


100%|██████████| 59/59 [00:31<00:00,  1.75it/s]

0.252846938685367
0.5052233537038167
0.42058566212654114
0.409000426530838


100%|██████████| 59/59 [00:31<00:00,  1.89it/s]


In [31]:
stat, p_value = mannwhitneyu(pipeline_scores, control_scores, alternative='two-sided')

print(f"U-statistic: {stat}")
print(f"P-value: {p_value}")

U-statistic: 2762.5
P-value: 3.841350845062791e-08
